# Regression of Used Car Prices Dataset
Run after following notebooks are run:
1. **01_Data_Cleaning**
2. **02_EDA**

## Development Notes/Ideas



## Libraries

In [1]:
import numpy as np
import pandas as pd
import optuna
from optuna_integration import LightGBMPruningCallback, CatBoostPruningCallback
from sklearn.model_selection import train_test_split
import sklearn.metrics

import lightgbm as lgb
import catboost as cb

## Load Data

In [2]:
train = pd.read_pickle('train_fe.pkl')
test = pd.read_pickle('test_fe.pkl')

## separate in to features and response variable
x_train = train.drop('price', axis=1)
y_train = train['price']

x_test = test

# package specific format
dataset_train = lgb.Dataset(x_train, label=y_train)

cols_cat = x_train.columns[x_train.dtypes=='category'].to_list()
pool_train = cb.Pool(x_train, label=y_train, cat_features=cols_cat)

## Optuna

### LightGBM

In [3]:
def objective_lgbm(trial):

    param = {
        "verbosity": 0,
        "objective": "regression",
        "metric": "rmse",
        "boosting": "gbdt",
        # learning rate
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
        # L1 regularization weight
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
        # L2 regularization weight
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
        # max leaves in one tree
        "num_leaves": trial.suggest_int("num_leaves", 10, 400),
        # subset of features on each tree
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        # randomly select part of data without resampling
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        # frequency for bagging, at every kth tree
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        # # minimum leaf weight, larger the term more conservative the tree
        # "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 100),
        # "feature_pre_filter": False,
        # maximum depth of the tree, signifies complexity of the tree.
        "max_depth": trial.suggest_int("max_depth", 4, 10, step=2),
    }

    # pruning to increase efficiency by stopping unpromising trials at early stages
    pruning_callback = LightGBMPruningCallback(trial, "valid rmse")
    # cross-validate and return mean of test scores
    lgbm = lgb.cv(train_set=dataset_train, params=param, num_boost_round=1000, nfold=5, stratified=True, seed=123, callbacks=[pruning_callback])
    score = lgbm["valid rmse-mean"][-1]
    return score

In [4]:
# specify sampler, seed and pruner
# note if optimizing a study in distributed or parallel mode, there is inherent non-determinism i.e. won't be able to reproduce results
sampler=optuna.samplers.TPESampler(seed=123) 
pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner, study_name='lgbm cv')
study.optimize(objective_lgbm, n_trials=300, timeout=10000, show_progress_bar=True, n_jobs=8)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-15 13:13:02,194] A new study created in memory with name: lgbm cv


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2024-09-15 13:15:24,127] Trial 0 finished with value: 76650.47519387968 and parameters: {'learning_rate': 0.07289789442461049, 'lambda_l1': 0.010565148600759506, 'lambda_l2': 6.279173555077436e-08, 'num_leaves': 95, 'feature_fraction': 0.5191083603789208, 'bagging_fraction': 0.5794621103245552, 'bagging_freq': 4, 'max_depth': 4}. Best is trial 0 with value: 76650.47519387968.
[I 2024-09-15 13:15:28,241] Trial 4 finished with value: 92034.47916793804 and parameters: {'learning_rate': 0.8204953156195421, 'lambda_l1': 1.969129015994745e-05, 'lambda_l2': 7.826422830908603e-08, 'num_leaves': 280, 'feature_fraction': 0.8676972463609514, 'bagging_fraction': 0.8402074358230696, 'bagging_freq': 7, 'max_depth': 4}. Best is trial 0 with value: 76650.47519387968.
[I 2024-09-15 13:17:44,986] Trial 9 finished with value: 73787.4213424163 and parameters: {'learning_rate': 0.00778449517406473, 'lambda_l1': 0.00010342203014953207, 'lambda_l2': 2.2878539917518365e-05, 'num_leaves': 101, 'feature_frac

### CatBoost

In [3]:
# split data in to train and validation set
train80_x, valid20_x, train80_y, valid20_y = train_test_split(x_train, y_train, test_size=0.2, random_state=123)
pool_train80 = cb.Pool(train80_x, label=train80_y, cat_features=cols_cat)
pool_valid20 = cb.Pool(valid20_x, label=valid20_y, cat_features=cols_cat)

In [4]:
def objective_catb(trial):

    param = {
        "verbose": False,
        "objective": "RMSE",
        "eval_metric": "RMSE",
        "boosting_type": "Plain",
        # learning rate
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
        # L2 regularization weight
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 1.0, log=True),
        # subset of features on each level
        "rsm": trial.suggest_float("rsm", 0.4, 1.0),
        # randomly select part of data without resampling
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        # frequency for bagging, at every kth tree
        # "sampling_frequency": trial.suggest_int("sampling_frequency", 1, 7),
        # # minimum leaf weight, larger the term more conservative the tree
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 100),
        # "feature_pre_filter": False,
        # maximum depth of the tree, signifies complexity of the tree.
        "max_depth": trial.suggest_int("max_depth", 4, 10, step=2),
        # Controls a way new nodes are added to the tree.
        "grow_policy": trial.suggest_categorical("grow_policy", ["Depthwise", "Lossguide"]),
    }

    if param["grow_policy"] == "Lossguide":
        # max leaves in one tree
        param["num_leaves"] = trial.suggest_int("num_leaves", 10, 400)

    # pruning to increase efficiency by stopping unpromising trials at early stages
    pruning_callback = CatBoostPruningCallback(trial, "RMSE")
    # cross-validate and return mean of test scores
    # catb = cb.cv(pool=pool_train, params=param, num_boost_round=1000, nfold=5, stratified=True, seed=123, callbacks=[pruning_callback])
    # score = catb['test-RMSE-mean'].iloc[-1]
    # return score

    # training and validating 
    catb = cb.CatBoostRegressor(**param, num_boost_round=1000, random_seed=123)
    catb.fit(pool_train80, eval_set=pool_valid20, callbacks=[pruning_callback])
    pruning_callback.check_pruned()
    preds = catb.predict(pool_valid20)
    score = sklearn.metrics.root_mean_squared_error(valid20_y, preds)
    return score

In [5]:
# specify sampler, seed and pruner
# note if optimizing a study in distributed or parallel mode, there is inherent non-determinism i.e. won't be able to reproduce results
sampler=optuna.samplers.TPESampler(seed=123) 
pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner, study_name='catb train80/valid20')
study.optimize(objective_catb, n_trials=300, timeout=10000, show_progress_bar=True, n_jobs=8)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-23 20:01:46,965] A new study created in memory with name: catb train80/valid20


  0%|          | 0/300 [00:00<?, ?it/s]

C:\Users\steph\AppData\Local\Temp\ipykernel_19476\3029905631.py:32: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2024-09-23 20:05:59,323] Trial 2 finished with value: 72604.9776743115 and parameters: {'learning_rate': 0.0020562003499205966, 'l2_leaf_reg': 0.00018500165575616873, 'rsm': 0.7247307814548334, 'subsample': 0.7885366077426945, 'min_data_in_leaf': 9, 'max_depth': 4, 'grow_policy': 'Depthwise'}. Best is trial 2 with value: 72604.9776743115.


C:\Users\steph\AppData\Local\Temp\ipykernel_19476\3029905631.py:32: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


...

[I 2024-09-23 22:43:29,423] Trial 221 pruned. Trial was pruned at iteration 53.
[I 2024-09-23 22:44:18,262] Trial 202 finished with value: 72045.75054872714 and parameters: {'learning_rate': 0.2517384053917069, 'l2_leaf_reg': 0.008321490943490138, 'rsm': 0.7404115131695014, 'subsample': 0.519295096140021, 'min_data_in_leaf': 56, 'max_depth': 4, 'grow_policy': 'Depthwise'}. Best is trial 113 with value: 71814.51598695361.


C:\Users\steph\AppData\Local\Temp\ipykernel_19476\3029905631.py:32: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2024-09-23 22:44:25,533] Trial 224 pruned. Trial was pruned at iteration 83.
[I 2024-09-23 22:44:35,609] Trial 227 pruned. Trial was pruned at iteration 8.
[I 2024-09-23 22:44:43,110] Trial 203 finished with value: 71976.61840621087 and parameters: {'learning_rate': 0.17617451677080048, 'l2_leaf_reg': 4.369292037610339e-05, 'rsm': 0.7455645503075065, 'subsample': 0.5148296478101808, 'min_data_in_leaf': 56, 'max_depth': 4, 'grow_policy': 'Depthwise'}. Best is trial 113 with value: 71814.51598695361.


C:\Users\steph\AppData\Local\Temp\ipykernel_19476\3029905631.py:32: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2024-09-23 22:44:44,515] Trial 226 pruned. Trial was pruned at iteration 28.
[I 2024-09-23 22:45:16,181] Trial 228 pruned. Trial was pruned at iteration 49.
[I 2024-09-23 22:45:24,746] Trial 231 pruned. Trial was pruned at iteration 5.
[I 2024-09-23 22:46:10,945] Trial 232 pruned. Trial was pruned at iteration 60.
[I 2024-09-23 22:46:53,646] Trial 233 pruned. Trial was pruned at iteration 53.
[I 2024-09-23 22:46:59,606] Trial 234 pruned. Trial was pruned at iteration 5.
[I 2024-09-23 22:47:06,703] Trial 235 pruned. Trial was pruned at iteration 5.
[I 2024-09-23 22:47:14,038] Trial 236 pruned. Trial was pruned at iteration 5.
[I 2024-09-23 22:48:55,490] Trial 210 finished with value: 71988.02616622795 and parameters: {'learning_rate': 0.09768435526172216, 'l2_leaf_reg': 2.1245695105512948e-05, 'rsm': 0.7144680691793627, 'subsample': 0.48900599572714515, 'min_data_in_leaf': 84, 'max_depth': 4, 'grow_policy': 'Lossguide', 'num_leaves': 198}. Best is trial 113 with value: 71814.51598695